This file generates the predicted final entries of each contest based on the models. 
For the scaled version, the final entries is the value of Ae^Bx when x is 100.
For the unscaled version, the final entries is the value of Ae^Bx when x is the duration of the contest.

In [1]:
import pandas as pd
import numpy as np

In [2]:
durationDF = pd.read_csv('data/Durations.csv').set_index("ContestId")

In [3]:
scaled = True

## Kalman AB Processing

In [4]:
abDF = pd.read_csv('data/KF_Values.csv').set_index("ContestId")
if('Unnamed: 0' in abDF.columns): ## I'm not sure why this column sometimes appears, so I just remove it
    abDF = abDF.drop(columns=['Unnamed: 0'])
df = pd.merge(durationDF, abDF, on='ContestId', how='left')
df = df.replace('TooShort', 0).replace('NaN', 0).replace(np.nan, 0)
if(scaled):
    df['Duration'] = 100

In [5]:
kfCols = []
for col in df.columns:
    if(col[0]=='A'):
        v = col[1:]
        newCol = 'Pred'+v
        print("Processing", v)
        df[newCol] = df['A'+v].astype(float)*np.exp(df['B'+v].astype(float)*df['Duration'])
        kfCols.append(newCol)

Processing v1
Processing v10
Processing v11
Processing v12
Processing v13
Processing v14
Processing v15
Processing v2
Processing v3
Processing v4
Processing v5
Processing v6
Processing v7
Processing v8
Processing v9


In [7]:
df.to_csv('data/KF_Preds.csv')

## Linear Regression AB Processing

In [8]:
LRDF = pd.read_csv("data/LR_Values.csv").set_index('ContestId')

In [9]:
if('Unnamed: 0' in LRDF.columns): ## I'm not sure why this column sometimes appears, so I just remove it
    LRDF = LRDF.drop(columns=['Unnamed: 0'])
df = pd.merge(durationDF, LRDF, on='ContestId', how='left')
df = df.replace('TooShort', 0).replace('HTooShort', 0)
if(scaled):
    df['Duration'] = 100

In [10]:
lrCols = []
for col in df.columns:
    if(col[::-1][0]=='A'):
        v = col[:-1]
        newCol = 'Pred'+v
        print("Processing", v)
        df[newCol] = df[v+'A'].astype(float)*np.exp(df[v+'B'].astype(float)*df['Duration'])
        lrCols.append(newCol)

Processing f10
Processing f11
Processing f12
Processing f13
Processing f14
Processing f15
Processing f1
Processing f2
Processing f3
Processing f4
Processing f5
Processing f6
Processing f7
Processing f8
Processing f9


In [12]:
df = df.replace(np.nan, 0.)

In [14]:
df.to_csv('data/LR_Preds.csv')